In [7]:
import PIL
from tkinter import *
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog
from tkinter import messagebox
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def upload():
    global panelA, panelB, panelC, image, grayimg, frame
    f_types = [("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")] 
    path = filedialog.askopenfilename(filetypes=f_types)
    image = cv2.imread(path) 
    image = cv2.resize(image, (500,500))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image1 = Image.fromarray(image)
    image1 = ImageTk.PhotoImage(image1)

    panelA = Label(frame, image=image1, borderwidth=5, relief="sunken")
    panelA.image = image1
    panelA.grid(row= 1, column=1 , rowspan= 13, columnspan= 3, padx=20, pady=20)
    
    return image

In [9]:
def grayscale(method=1, shades=256):
    global image, grayimg, frame  # Pastikan image sudah didefinisikan sebelumnya
    img_array = np.array(image).astype(np.int32)  # Ubah ke int32 untuk mencegah overflow
    height, width, _ = img_array.shape
    grayimg = np.zeros((height, width), dtype=np.uint8)
    
    for i in range(height):
        for j in range(width):
            r, g, b = img_array[i, j]
            
            if method == 1: # Metode 1: Rata-rata (Average)
                gray = (r + g + b) / 3  
            elif method == 2: # Metode 2: Perceptual Grayscale
                gray = int(r * 0.299 + g * 0.587 + b * 0.114)
            elif method == 3: # Metode 3: Luminosity ((Max + Min)2)
                gray = (max(r, g, b) + min(r, g, b)) // 2
            elif method == 4: # Metode 4: Komponen Tunggal (Merah)
                gray = r  
            elif method == 5: # Metode 5: Komponen Tunggal (Hijau)
                gray =g   
            elif method == 6: # Metode 6: Komponen Tunggal (Biru)
                gray = b
            elif method == 7: # Metode 7: Max Komponen
                gray = max(r, g, b)
            elif method == 8: # Metode 8: Min Komponen
                gray = min(r, g, b)
            elif method == 9: # Metode 9: Conversion Factor (255 Shades)
                conversion_factor = 255 / (shades - 1)
                avg_value = (r + g + b) / 3
                gray = int((avg_value / conversion_factor) + 0.5) * conversion_factor
            
            grayimg[i, j] = np.clip(gray, 0, 255)  # Hindari nilai lebih dari 255
    
    grayimg1 = Image.fromarray(grayimg)
    grayimg1 = ImageTk.PhotoImage(grayimg1)
    
    panelB = Label(frame, image=grayimg1, borderwidth=5, relief="sunken")
    panelB.image = grayimg1
    panelB.grid(row=1, column=4, rowspan=13, columnspan=3, padx=20, pady=20)
    
    return grayimg


In [10]:
grayimg = None 

def histogram():
    global grayimg, panelC, frame
    if grayimg is None or grayimg.size == 0:
       messagebox.showerror("Error", "Gambar grayscale belum dibuat! Silakan klik tombol GRAYSCALE terlebih dahulu.")
       return

    histogram, _ = np.histogram(grayimg.flatten(), bins=256, range=[0,256])

    # Simpan histogram sebagai gambar
    plt.figure(figsize=(6, 3))
    plt.bar(range(256), histogram, color='gray')
    plt.title("Histogram Grayscale")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.savefig("histogram.png", bbox_inches='tight')
    plt.close()

    # Tampilkan histogram di GUI
    hist_img = Image.open("histogram.png")
    hist_img = hist_img.resize((500, 230))  # Resize agar muat di GUI
    hist_tk = ImageTk.PhotoImage(hist_img)

    panelC = Label(frame, image=hist_tk, borderwidth=5, relief="sunken")
    panelC.image = hist_tk
    panelC.grid(row=14, column=1, columnspan=6, padx=20, pady=20)  # Posisi di bawah gambar

    


In [ ]:
root = Tk()
root.title("IMAGE PROCESSING")

# Canvas untuk membuat tampilan bisa di-scroll
canvas = Canvas(root)
canvas.pack(side=LEFT, fill=BOTH, expand=True)

# Scrollbar vertikal
scrollbar = Scrollbar(root, orient=VERTICAL, command=canvas.yview)
scrollbar.pack(side=RIGHT, fill=Y)

# Frame di dalam Canvas
frame = Frame(canvas)
canvas.create_window((0, 0), window=frame, anchor="nw")

# Konfigurasi agar scrollbar berfungsi
def update_scroll_region(event):
    canvas.configure(scrollregion=canvas.bbox("all"))

frame.bind("<Configure>", update_scroll_region)
canvas.configure(yscrollcommand=scrollbar.set)

# Fungsi agar bisa scroll pakai mouse wheel
def on_mouse_wheel(event):
    canvas.yview_scroll(-1 * (event.delta // 120), "units")

root.bind_all("<MouseWheel>", on_mouse_wheel)  

# ======= Tombol-tombol =======
l1 = Label(frame, text="Tekan tombol untuk menjalankan fungsi yang tertulis",
           fg="white", bg="purple", width=98, borderwidth=5, relief="groove", font=('Verdana', 15))
l1.grid(row=0, column=1, columnspan=6, padx=20, pady=20, sticky='nesw')

btn_upload = Button(frame, text="UPLOAD", fg="black", bg="lavender", command=upload)
btn_upload.grid(row=1, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Rata-rata", fg="white", bg="snow4", command=lambda : grayscale(1))
btn_grayscale.grid(row=2, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Perceptual Grayscale", fg="white", bg="snow4", command=lambda : grayscale(2))
btn_grayscale.grid(row=3, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="Luminosity)", fg="white", bg="snow4", command=lambda : grayscale(3))
btn_grayscale.grid(row=4, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Komponen Tunggal (Merah)", fg="white", bg="snow4", command=lambda : grayscale(4))
btn_grayscale.grid(row=5, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Komponen Tunggal (Hijau)", fg="white", bg="snow4", command=lambda : grayscale(5))
btn_grayscale.grid(row=6, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Komponen Tunggal (Biru)", fg="white", bg="snow4", command=lambda : grayscale(6))
btn_grayscale.grid(row=7, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Max Komponen", fg="white", bg="snow4", command=lambda : grayscale(7))
btn_grayscale.grid(row=8, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Min Komponen", fg="white", bg="snow4", command=lambda : grayscale(8))
btn_grayscale.grid(row=9, column=0, padx=10, pady=10, sticky='nesw')

btn_grayscale = Button(frame, text="GRAYSCALE-Conversion Factor (255 Shades)", fg="white", bg="snow4", command=lambda : grayscale(9))
btn_grayscale.grid(row=10, column=0, padx=10, pady=10, sticky='nesw')

btn_histogram = Button(frame, text="HISTOGRAM", fg="black", bg="lightblue", command=histogram)
btn_histogram.grid(row=11, column=0, padx=10, pady=10, sticky='nesw')

In [12]:
root.mainloop()